In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
# https://towardsdatascience.com/lstms-in-pytorch-528b0440244
# https://github.com/IHCA-Coding-to-Prevent-Coding/ML-SampleNeuralNet/blob/main/sampleneuralnet.py
# data from https://archive.ics.uci.edu/ml/datasets/Power+consumption+of+Tetouan+city#

#testing data
# data format is date/time, temperature, humidity, wind speed, general diffuse flow, diffue flow,  zone 1 power, 2 power, 3 power
with open("Tetuan City power consumption.csv", 'r') as f:
    rawData = f.readlines()

    #removing head
    rawData = rawData[1:len(rawData)]
    
def breakLines(string):
    itemizedLine=[]
    prev=0
    for c in range(len(string)):
        if string[c] == str(','):
            itemizedLine.append(string[prev:c])
            prev=c
    return itemizedLine

itemizedLines=[]
for line in rawData:
    itemizedLines.append(breakLines(line))

#TODO convert to pandas
#TODO format neural input object
#TODO make sure neural accepts info

# takes 144 inputs (24 hours) for each data type (temp, humidity, windSpeed, general diffuse, and diffuse)
workingLines = pd.DataFrame(itemizedLines[0:145*14]) # gets 2 weeks of weather data
# ['1/1/2017 0:00', ',6.559', ',73.8', ',0.083', ',0.051', ',0.119', ',34055.6962', ',16128.87538']

alltemps = [line[1] for line in workingLines]
allhums = [line[2] for line in workingLines]
allwinds = [line[3] for line in workingLines]
allgendif = [line[4] for line in workingLines]
alldif = [line[5] for line in workingLines]

testing = [alltemps, allhums, allwinds]

In [ ]:
#testing = torch.randn(batchSize, seqLength, inputSize)

inputSize=5 # variables input
hiddenSize=1 # nodes of hidden layer
outSize=1 # power consumptoion in zone 1
batchSize=100 # how many data points are in batch
lr = 0.1
seqLength=144 # num of time values per input

class extractTensor(nn.Module):
    def forward(self, x):
        tensor, hs = x
        return tensor.reshape(-1, hiddenSize) 

model = nn.Sequential(
    nn.LSTM(inputSize, hiddenSize),
    extractTensor(),
    nn.Linear(hiddenSize, outSize)
)

def lossFunc(date=testing, comparison=truth, model=model):
    prediction = model(testing)

    MSEloss = torch.nn.MSELoss()
    output = MSEloss(prediction, comparison)
    
    return output

optimizer = torch.optim.SGD(model.parameters(), lr=lr)

lossList = []
for i in range(100):
    loss = lossFunc()
    lossList.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

plt.plot(lossList)

In [3]:
# training data digits

from sklearn.datasets import load_digits

digits = load_digits(return_X_y=True)
#names = load_targets()
print(digits)
plt.show()

(array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]]), array([0, 1, 2, ..., 8, 9, 8]))
